# MNE INV

The purpose of this notebook is to see if MNE-Python will play nicely with our current inputs such that we can use it to compute the inverse solution.

In [12]:
from EMGinv_fns import tmsi_eventextractor, load_tmsi_data, fwd_convertfixed, _apply_inverse_no_reference_check, bone_remover, src_bone_remover, fwd_bone_remover
from geometry_utilities import *
from plotter_utility_functions import *

import mne
import mne.minimum_norm.inverse # Sometimes Python / Lazy loading can't find the module, so import explicitly here (required to use _apply_inverse_no_reference_check)
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
import scipy.io

import pyvistaqt
%matplotlib qt

In [13]:
# %load_ext autoreload
# %autoreload 2

# Processing Flags #
Set flags/environment-like-variables here, which will be used in rest of the notebook to determine what to process and how.

In [14]:
NUM_ELECTRODES = 256  # Noting that I only have the 256 eletrode configuration currently.  
# ARM = 'left'
# NUM_ELECTRODES = 128
ARM = 'right'

# MIN_DIPOLE_DISTANCE = 7.0 #mm
MIN_DIPOLE_DISTANCE = 2.0 #mm

SAMPLE_RATE = 2000. # samples per second (from MUAPs file)

# MASK_FILE = f'Data/{EXPERIMENT}_muaps_mask.mat'
# TEMPLATES_FILE = f'Data/{EXPERIMENT}_muaps_template-{MUAP_ID:02d}.mat'
# COVARIANCE_FILE = f'Data/{EXPERIMENT}_muaps_covariance.mat'
ARM_DICOM_FILENAME = 'Data/R_Forearm.dcm'

ELECTRODES_FILE = f'Data/{NUM_ELECTRODES}simparm_electrode_pos.npy'
MNE_FWD_SOLUTIONS_FILE = f'Data/simp_arm_2mm-fwd.fif'
# EXTRACT_MUAPS_FROM_MASK = False;

# Extra parameters for source inversion
FIXED_ORIENT = False # Fixed orientation to be True or False 
SET_AVG_REF = True # Set average reference to be True or False. MNE-Python recommmends True as a projection, but False is required for a custom reference.  Will also bypass an MNE-Python check.
REG_PARAM = 0.05  # MNE-Python recommended are: 0.05 for LCMV Beamformer and 1/9 for the other methods
INV_METHOD = 'LCMV' # Options are: 'MNE', 'dSPM', 'eLORETA', 'sLORETA', 'LCMV'
LCMV_pick_ori = None # Parameter only for LCMV: None, 'normal', 'max-power', 'vector'
LCMV_weight_norm = 'unit-noise-gain-invariant' # Parameter only for LCMV: None, 'unit-noise-gain', 'nai', 'unit-noise-gain-invariant'
LCMV_depth = None # Parameter only for LCMV: None, float (None is default).  Normalises the leadfield # https://mne.discourse.group/t/mne-analysis-digest-vol-142-issue-20/2006

In [15]:
# Loading source space and forward model generated in MNE-python, see Fwd_BEM_MNE.ipynb
# Load electrode positions
electrode_pos = np.load(ELECTRODES_FILE)

# Load forward model
mne_fwd = mne.read_forward_solution(MNE_FWD_SOLUTIONS_FILE)
xscaling, yscaling, zscaling = (MIN_DIPOLE_DISTANCE*1e-3, MIN_DIPOLE_DISTANCE*1e-3, MIN_DIPOLE_DISTANCE*1e-3) # Is the distance between dipoles in source space #np.repeat(np.abs(np.sum(np.diff(pos[0], axis = 0))),3)  

# Transformations specific to dataset"
# Need to remove some electrodes - get rid of the middle ones
electrode_pos = np.vstack((electrode_pos[:64,:], electrode_pos[192:,:]))

# Removal of dipole sources - here remove sources lying within cylinders representing bone - it may be preferable to generate a fwd model with the sources removed in Fwd_BEM_MNE.ipynb instead
mne_fwd = fwd_bone_remover(mne_fwd, -9e-3, 0, 5e-3, inplace=False) # Ulnar
mne_fwd = fwd_bone_remover(mne_fwd, 5e-3, -9e-3, 5e-3, inplace=False) # Radius
pos  = mne_fwd['source_rr'] 

# Condense the fwd such that there is only one dipole per voxel
# If want to specify what orientation this is, need to adjust what is defined as the normal in the source space object for each source location. 
# E.g. mne_fwd['src'][0]['nn'] = np.tile(np.array([0,0,1]), (mne_fwd['src'][0]['nn'].shape[0],1))
if FIXED_ORIENT:
    mne.convert_forward_solution(mne_fwd, surf_ori=True, force_fixed=True, use_cps=False, copy=False)

Reading forward solution from /Users/pokhims/Library/CloudStorage/OneDrive-TheUniversityofMelbourne/Documents/Coding/CMU_EMGSL/Data/simp_arm_2mm-fwd.fif...
    Reading a source space...
    [done]
    1 source spaces read
    Desired named matrix (kind = 3523 (FIFF_MNE_FORWARD_SOLUTION_GRAD)) not available
    Read EEG forward solution (23424 sources, 256 channels, free orientations)
    Source spaces transformed to the forward solution coordinate frame


In [16]:
def load_tmsitomne_combine(f_prox=None, f_dist=None, scale=1e-6):
    """ Combines the distal and proximal datasets from TMSI together into one MNE object.
    
    Parameters: 
    - f_prox (str): the path to the proximal file to be loaded.
    - f_dist (str): the path to the distal file to be loaded.
    - scale (float): the scaling factor.  By default it will convert the data from uV to V.

    Returns:
    - MNE_raw (MNE raw object): the combined MNE object.
    """
    if f_prox is None:
        f_prox = 'Data/Pok_2024_08_21_A_PROX_8.poly5'
    if f_dist is None:
        f_dist = 'Data/Pok_2024_08_21_B_DIST_8.poly5' 

    prox = load_tmsi_data(filename=f_prox)
    dist = load_tmsi_data(filename=f_dist)      
    # ch_names = ["Prox - " + s for s in prox[1]] + ["Dist - " + s for s in dist[1]]
    # Need to make sure the ch_names between the fwd and dataset are aligned.
    ch_names = mne_fwd.ch_names[:64] + ['Prox - AUX 1-1', 'Prox - AUX 1-2', 'Prox - AUX 1-3','Prox - TRIGGERS', 'Prox - STATUS', 'Prox - COUNTER',] +  mne_fwd.ch_names[-64:] + ['Dist - TRIGGERS', 'Dist - STATUS', 'Dist - COUNTER']
    fs = prox[2]
    ch_types = ['eeg']*64 + ['misc']*(len(prox[1])-64) + ['eeg']*64 + ['misc']*(len(dist[1])-64)

    num_channels = len(ch_names)
    dist_sec = np.where(dist[0][-3,:]==254)[0]
    prox_sec = np.where(prox[0][-3,:]==254)[0]

    dist_sample = dist[0][:, dist_sec[0]:dist_sec[-1]]
    prox_sample = prox[0][:, prox_sec[0]:prox_sec[-1]]

    # Scale the data channels
    dist_sample[:64] = dist_sample[:64]*scale
    prox_sample[:64] = prox_sample[:64]*scale

    del dist, prox
    # Create MNE raw object
    info = mne.create_info(ch_names, fs, ch_types)
    # The lengths of the two datasets are stil off, so remove some samples, still misaligned slightly
    MNE_raw = mne.io.RawArray(np.concatenate((prox_sample[:,1:-1], dist_sample), axis=0), info)

    return MNE_raw

In [17]:
# Load some data - to construct covariance matrices
MNE_raw = load_tmsitomne_combine(f_prox = 'Data/Pok_2024_08_21_A_PROX_8.poly5', f_dist='Data/Pok_2024_08_21_B_DIST_8.poly5', scale=1e-6)

Reading file  Data/Pok_2024_08_21_A_PROX_8.poly5
	 Number of samples:  892710 
	 Number of channels:  70 
	 Sample rate: 4000 Hz
Done reading data.
Creating RawArray with float64 data, n_channels=70, n_times=892710
    Range : 0 ... 892709 =      0.000 ...   223.177 secs
Ready.
Sample rate:  4000  Hz
Channel names:  ['UNI 01', 'UNI 02', 'UNI 03', 'UNI 04', 'UNI 05', 'UNI 06', 'UNI 07', 'UNI 08', 'UNI 09', 'UNI 10', 'UNI 11', 'UNI 12', 'UNI 13', 'UNI 14', 'UNI 15', 'UNI 16', 'UNI 17', 'UNI 18', 'UNI 19', 'UNI 20', 'UNI 21', 'UNI 22', 'UNI 23', 'UNI 24', 'UNI 25', 'UNI 26', 'UNI 27', 'UNI 28', 'UNI 29', 'UNI 30', 'UNI 31', 'UNI 32', 'UNI 33', 'UNI 34', 'UNI 35', 'UNI 36', 'UNI 37', 'UNI 38', 'UNI 39', 'UNI 40', 'UNI 41', 'UNI 42', 'UNI 43', 'UNI 44', 'UNI 45', 'UNI 46', 'UNI 47', 'UNI 48', 'UNI 49', 'UNI 50', 'UNI 51', 'UNI 52', 'UNI 53', 'UNI 54', 'UNI 55', 'UNI 56', 'UNI 57', 'UNI 58', 'UNI 59', 'UNI 60', 'UNI 61', 'UNI 62', 'UNI 63', 'UNI 64', 'AUX 1-1', 'AUX 1-2', 'AUX 1-3', 'TRIGGER

In [18]:
# Important to filter the data (highpass filter)
MNE_raw = MNE_raw.filter(l_freq=100, h_freq=None, )
# Estimate the noise covariance matrix on Epoched data.  This means that the noise covariance will be estimated on pre-stimulus periods
channel_data = MNE_raw['Prox - TRIGGERS'][0][0]-252
events = tmsi_eventextractor(channel_data)
event_dict = {'Ext': -2, 'Flex': -6} # Should be correct 
epochs = mne.Epochs(MNE_raw, events, event_dict, tmin=-4, tmax=4, baseline=None, preload=True)
# Consider setting an average EEG reference across each panel of 32. 
# epochs.set_eeg_reference('average', projection=True)
# epochs.apply_proj()
# epochs.plot(n_epochs=1, scalings='auto', );

# Set tmin and tmax based on experimental conditions.  In this case, the participant was not moving perfectly to the triggers.
noise_cov = mne.compute_covariance(epochs, method='auto', tmin=-2, tmax=0.01)
data_cov = mne.compute_covariance(epochs, method='auto', tmin=1, tmax=epochs.tmax)

noise_cov.plot(epochs.info,)
data_cov.plot(epochs.info,)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 100.00
- Lower transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 87.50 Hz)
- Filter length: 529 samples (0.132 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.9s


Not setting metadata
20 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 20 events and 32001 original time points ...
0 bad epochs dropped
Reducing data rank from 128 -> 128
Estimating covariance using SHRUNK
Done.
Estimating covariance using DIAGONAL_FIXED
    EEG regularization : 0.1
Done.
Estimating covariance using EMPIRICAL
Done.
Using cross-validation to select the best estimator.
    EEG regularization : 0.1
    EEG regularization : 0.1
    EEG regularization : 0.1
Number of samples used : 160820
log-likelihood on unseen data (descending order):
   empirical: -305.882
   shrunk: -306.103
   diagonal_fixed: -313.113
selecting best estimator: empirical
[done]
Reducing data rank from 128 -> 128
Estimating covariance using SHRUNK
Done.
Estimating covariance using DIAGONAL_FIXED
    EEG regularization : 0.1
Done.
Estimating covariance using EMPIRICAL
Done.
Using cross-validation to select the best estimator.
    EEG r

(<Figure size 380x370 with 2 Axes>, <Figure size 380x370 with 1 Axes>)

In [19]:
# Sample waveforms to play with - Replace this cell with the time series array of the EMG data

# Alternative to muaps_from matlab.mat - Load the mask, and then extract from the relevant EMG channels in MNE_raw
filename = '/Users/pokhims/Library/CloudStorage/OneDrive-TheUniversityofMelbourne/Documents/Coding/CMU_EMGSL/Data/muaps_mask.mat'
mask = scipy.io.loadmat(filename)['mask']
# Not sure why some of the masks aren't in the data - and why the end result is so different
mask = mask[:-2, :]
data = MNE_raw.get_data(picks='data')
# Need to get component for all channels
muaps = np.zeros((128,41))
for i in range(128):
    tmp = data[i,:]
    snips = tmp[mask]
    muaps[i,:] = np.mean(snips, axis=0)

# Another alternative to the data
ext_1 = epochs.get_data(picks='data')[0,:,25030:25070]
flex_1 = epochs.get_data(picks='data')[0,:,26670:26710]

# Specify the waveform to be used
waveform = ext_1

In [20]:
# Load the waveform into an MNE Evoked Object - Typically evoked data is averaged over multiple trials, but this is not a strict resitriction.  
# Likely do the pre-processing as you please before using MNE. 

# info = mne.create_info(ch_names, fs, ch_types) # If you need to create info object from scratch, here is a hint

# Modify info object to only include data channels
datach_indices = mne.pick_types(epochs.info, eeg=True, )
info = mne.pick_info(epochs.info, datach_indices, copy=True)
evoked = mne.EvokedArray(waveform, info)

# MNE Python requires the use of an average reference to be set using it's projectors to do source inversion
if SET_AVG_REF:
    evoked.set_eeg_reference('average', projection=True)

EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.


In [21]:
if INV_METHOD == 'LCMV':
    filters = mne.beamformer.make_lcmv(
        evoked.info,
        mne_fwd,
        data_cov,
        reg=REG_PARAM,
        noise_cov=noise_cov,
        pick_ori=LCMV_pick_ori,
        weight_norm=LCMV_weight_norm,
        depth=LCMV_depth,
        rank=None,
    )
    stc_est_region = mne.beamformer.apply_lcmv(evoked, filters)

else:
    if SET_AVG_REF:
        # If data is already referenced, and do not want to use average, will need to replace MNE inverse function with a custom function to bypass this reference check in the inversion step. 
        mne.minimum_norm.inverse._apply_inverse = _apply_inverse_no_reference_check
    inverse_operator = mne.minimum_norm.make_inverse_operator(evoked.info, mne_fwd, noise_cov, loose='auto', depth=None, fixed=FIXED_ORIENT) 
    stc_est_region = mne.minimum_norm.apply_inverse(evoked, inverse_operator, REG_PARAM, INV_METHOD, pick_ori=None)

Computing rank from covariance with rank=None
    Using tolerance 3.6e-13 (2.2e-16 eps * 128 dim * 13  max singular value)
    Estimated rank (eeg): 127
    EEG: rank 127 computed from 128 data channels with 1 projector
Computing rank from covariance with rank=None
    Using tolerance 1.7e-13 (2.2e-16 eps * 128 dim * 5.9  max singular value)
    Estimated rank (eeg): 127
    EEG: rank 127 computed from 128 data channels with 1 projector
Making LCMV beamformer with rank {'eeg': 127}
Computing inverse operator with 128 channels.
    128 out of 256 channels remain after picking
Selected 128 channels
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 1)
Computing rank from covariance with rank={'eeg': 127}
    Setting small EEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computing beamformer filters for 20654 sources
Filter computation complete
combining the current components...


In [22]:
# Graph using our functions

# Threshold
thresh = 0.5
# Look at specific timepoint in the source activity:
t = 4

source_activity = stc_est_region.data[:, t]

# Reshape source activity to condense N source orientations into 1 per voxel - Confirmed works for 3 orientations, should work for more.
reshape_by = source_activity.shape[0] // pos.shape[0]
reshaped_act = np.array(source_activity.reshape((reshape_by, -1), order='F'))
source_activity = np.linalg.norm(reshaped_act, axis=0)

ind = np.abs(source_activity) > thresh*np.max(np.abs(source_activity))
source_activity = source_activity[ind]
pos_t = pos[ind]

# Plot the convex hull and the moved points
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
# Plot electrode positions
ax.scatter(electrode_pos[:, 0], electrode_pos[:, 1], electrode_pos[:, 2], c=waveform[:,t], marker='o', alpha=0.2, cmap='turbo')
# Plot the source space
ax.scatter(pos_t[:, 0], pos_t[:, 1], pos_t[:, 2], c=source_activity, marker='s', alpha=0.8, cmap='viridis')
# Set labels
ax.set_xlabel('X Axis (m)')
ax.set_ylabel('Y Axis (m)')
ax.set_zlabel('Z Axis (m)')
n_channels, n_sources = mne_fwd['sol']['data'].shape
ax.set_title(f'{n_channels}-Channel {n_sources}-{INV_METHOD} Method')

# # Save the figure to a file in the created folder
# plt.savefig(f'{save_dir}/{EXPERIMENT}_3D-T{t}.png', dpi=300)

Text(0.5, 0.92, '256-Channel 61962-LCMV Method')